# Import

In [ ]:
## LIBRARY IMPORT STATEMENTS ##

# Basic Data Libraries
import numpy as np
import pandas as pd
import json

# Plotting and Visualization Libraries
from matplotlib import pyplot as plt        # Package for graph creation

# Utility Libraries
import tkinter as tk                # Standard GUI library
from tkinter import filedialog      # Allows pop-up file open dialog
from tqdm.notebook import tqdm      # Package for progress bars
from datetime import datetime
import collections
import time

# System Libraries
import os
import sys
import subprocess
from os import path
sys.path.append('../.')     # Add the parent folder of the current script to the python search list

# Specialty Libraries
import cantera as ct    # Open source library for chemical kinetics, thermodynamics,
                        # and transport problems.

# Parameter Definitions

In [ ]:
## GLOBAL PARAMETER DEFINITIONS ##

# Thermodynamic Parameters
Pi = 101000 # Initial pressure Pa
Ti = 300 # Initial unburned gas temperature K

# Species Lists
air = {'N2': 3.76, 'O2': 1}     # Standard air composition

gas_list_ox = ['O2']

gas_list_inert = ['H2O',
                  'CO2',
                  'N2'
                  ]

gas_list_fuel = ["CO",
                "H2",
                "CH4",
                "C2H4",
                "C2H6",
                "C3H8",
                ]

gas_list_full = ['CO2', 'CO','N2', 'O2', 'H2O', 'H2', 'CH4', 'C2H4', 'C2H6', 'C3H8'] # Legacy to be removed and replaced in all instances with
                                                                                     # 'gas_list_all'

gas_list_all = gas_list_inert + gas_list_fuel + gas_list_ox

# Useful dictionaries for shifting back and forth between written names and formulas
name_formula_all = {'Carbon Dioxide': 'CO2',
                    'Carbon Monoxide': 'CO',
                    'Hydrogen': 'H2',
                    'Methane': 'CH4',
                    'Ethylene': 'C2H4',
                    'Ethane': 'C2H6',
                    'Propylene': 'C3H6',   #NOT IN GRIMECH3.0
                    'Propane': 'C3H8'}

formula_name_all = {'CO2': 'Carbon Dioxide',
                    'CO': 'Carbon Monoxide',
                    'H2': 'Hydrogen',
                    'CH4': 'Methane',
                    'C2H4': 'Ethylene',
                    'C2H6': 'Ethane',
                    'C3H6': 'Propylene',    #NOT IN GRIMECH3.0
                    'C3H8': 'Propane',
                    }

# Function Definitions

In [ ]:
## FUNCTION DEFINITIONS ##

def select_file():

    '''
    A quick function to create a file dialog window to prompt the user. Uses tkinter and
    includes code to make the dialog on top and test if something was selected.

    Arguments: None

    Returns:

        selection --> [str] a string with the path to the selected file
    '''
    selection = None    # Initialize a selection variable

    initial_directory = 'Data_Files'

    # FORCES THE DIALOG TO BE ON THE TOP RATHER THAN BENEATH ALL OTHER WINDOWS
    root = tk.Tk()
    root.attributes('-topmost', 1) #FORCE FILEDIALOG TO OPEN ON TOP
    root.withdraw()

    # GENERATE FILE DIALOG WINDOW MAKING .CSV AND .XLSX FILES VISIBLE FOR SELECTION
    selection = filedialog.askopenfilename(title='Select the file', filetypes=[('CSV Files,', '*.csv')], initialdir=initial_directory)

    # TEST IF SOMETHING WAS SELECTED
    if not selection:

        raise Exception('No file selected. Please try again.')

    return selection

##########################################################################################################
##########################################################################################################
def lower_limit(mix, oxidizer):

    '''
    This function is designed to calculate the Lower Flammable Limit (LFL) for a specified fuel gas
    mixture when burned with a given oxidizer.

    It does this by first having Cantera find the molar amounts Xf and Xa corresponding to an equivalence ratio (phi)
    of 1. It then sets this Xf as the upper bound of a binary search algorithm.

    Pick the halfway point of Xf range --> Is it flammable? --> If yes, divide Xf in half, pick the middle of the lower half, repeat.
                                                            --> If no, divide Xf in half, pick the middle of the upper half, repeat.
                                                            --> Stop when the Xf increment drops below a specified threshold

    Arguments:
        fuel -->    [dict] contains the composition information for the fuel species in question

        oxidizer --> [dict] contains the composition information for the oxidizer composition (i.e. air or O2)

    Returns:
        LFL --> [float] containing the lowest tested molar ratio of fuel gas to air that will burn

    '''

     # Initialize Cantera objects
    carbon = ct.Solution('graphite.yaml')        # Create carbon phase object to represent soot
    gas1 = ct.Solution('gri30.yaml')             # Create gas phase object to hold species of interest
    # gas1.X = fuel            # Set the composition of the gas phase object
    # gas1.TP = Ti, Pi         # Set the temperature and pressure of the gas phase object to ambient

    phi = 1.0               # Set starting equivalence ratio value
    loopcount = 0

    alpha = list(mix.values())     # Generate ordered list of individual mole fractions
    Tui = [df_TempCrit.loc[key]['Tu'] for key in list(mix.keys())]     # Generate list of individual lower critical temperatures
    Tli = [df_TempCrit.loc[key]['Tl'] for key in list(mix.keys())]     # Generate list of individual upper critical temperatures
    Tl = np.dot(alpha, Tli)     # Generate weighted average critical temperature for lower limit
    Tu = np.dot(alpha, Tui)     # Generate weighted average critical temperature for upper limit

    gas1.set_equivalence_ratio(phi, mix, oxidizer)      # Have Cantera automatically set the equivalence ratio to 1.0

    Xf_stoich = 1 - sum(gas1['O2','N2'].X)     # Subtract oxygen and nitrogen (air) to find the molar fuel fraction
    Yf_stoich = 1 - sum(gas1['O2','N2'].Y)     # Subtract oxygen and nitrogen (air) to find the mass fuel fraction

    phi_lfl = phi       # Initialize phi for lower flammability limit
    Xf_lfl = Xf_stoich  # Initialize Xf for lower flammability limit

    flag = False        # Initialize flammability limit found flag

    # Start binary search
    while (flag != True):
        gas1.set_equivalence_ratio(phi, mix, oxidizer)

        print(phi)
        print(loopcount)
        loopcount += 1

        phi_lfl = phi
        Xf_lfl = 1 - sum(gas1['O2','N2'].X)     # Subtract oxygen and nitrogen (air) to find the molar fuel fraction
        Yf_lfl = 1 - sum(gas1['O2','N2'].Y)     # Subtract oxygen and nitrogen (air) to find the mass fuel fraction

        gas1.equilibrate('HP')
        t_eq = gas1.T
        d_phi = (phi/2)
        if t_eq >= Tl:

            phi = phi - d_phi

        if t_eq < Tl:

            phi = phi + d_phi

        if d_phi < 0.01:

            flag = True


    return Xf_lfl, Yf_lfl, phi

#######################################################################################################
#######################################################################################################

def Average_Limits(mix):

    '''
    This function implements a form of Le Chatelier's method for estimating flammablility limits.
    In escence it is simply a weighted average of the flammaiblity limits of the individual component gases.

    Limitations:
        ---> Originally conceived specifically for hydrocarbon mixtures
        ---> Originally conceived mostly for LFL
        ---> Can be used for UFL, but accuracy suffers
        ---> Specifically questionable for mixes with significant hydrogen

    Note: As of 11/16/2023 the values are tabulated in the included Excel document for combustion in AIR ONLY

    Arguments:
        mix --> [dict] a dictionary with the compositions for the mixture

    Returns:
        Xf_lfl --> [float] with the lower limit in mole fraction (0) and mass fraction (1)
        Xf_ufl --> [float] with the upper limit in mole fraction (0) and mass fraction (1)
    '''

    key_list = list(mix.keys())
    value_list = list(mix.values())

    Tui = [df_limits.loc[key]['X_upper'] for key in key_list]
    Tli = [df_limits.loc[key]['X_lower'] for key in key_list]

    Xf_lfl = np.dot(value_list,Tli)     # Dot product to calculate the weighted average LFL of the components
    Xf_ufl = np.dot(value_list, Tui)    # Dot product to calculate the weighted average UFL of the components

    return Xf_lfl, Xf_ufl


##############################################################################################################
##############################################################################################################

def Tblend(mixx):

    '''
    This function takes a dictionary with a list of FUEL gases and their compositions. It then
    calculates the critical ignition temperatures by doing a weighted sum of the critical temperatures
    for the individual component fuel gases.

    Arguments:
        mix --> [dict] containing the MOLAR fractions of the individual fuel gases in the fuel mixture

    Returns:
        Tl --> [float] lower limit critical temperature
        Tu --> [float] upper limit critical temperature
    '''

    alpha = list(mixx.values())     # Generate ordered list of individual mole fractions
    Tui = [df_TempCrit.loc[key]['Tu'] for key in list(mixx.keys())]     # Generate list of individual lower critical temperatures
    Tli = [df_TempCrit.loc[key]['Tl'] for key in list(mixx.keys())]     # Generate list of individual upper critical temperatures
    Tl = np.dot(alpha, Tli)     # Generate weighted average critical temperature for lower limit
    Tu = np.dot(alpha, Tui)     # Generate weighted average critical temperature for upper limit
    return Tl, Tu

################################################################################################################
################################################################################################################

def Flamespeed(mix, Ti, Pi):

    '''
    This function takes in a dictionary with the overall component fractions. (i.e. not the fraction of fuel overall Xf or air Xa,
    but the individual species. It uses a solver to solve for the flame speed.

    Arguments:
        mix --> [dict] with the individual species of the overall mixture (including air species {O2 and N2}) at the desired
                equivalence ratio.

                Example -- {'CO': 0.2, 'CH4': 0.25, 'O2': 0.1, 'N2: 0.376}

        Ti -->  [float] containing the temperature in [K] at which the flame speed should be calculated
                Defaults to 300 K

        Pi -->  [float] containing the pressure in [Pa] at which the flame speed should be calculated
                Defaults to 101325 Pa

    Returns:
        speed --> [float] containing the laminar flame speed Su in [m/s]
    '''

    try:
        gas2 = ct.Solution('gri30.yaml')
        gas2.X = mix
        gas2.TP = Ti, Pi

        #Laminar Flame Speed
        f = ct.FreeFlame(gas2, width=5) #A freely-propagating flat flame
        #Energy equation enabled
        f.energy_enabled = True
        f.set_max_time_step(50000)
        f.set_refine_criteria(ratio = 2.0, slope = 0.05, curve = 0.05)
        f.transport_model = 'Mix' # Solve with multi or mix component transport properties
        f.solve(loglevel=1,auto=True,refine_grid=True) #
        print('Flamespeed Done! ', f.velocity[0])
        speed = f.velocity[0]
        return speed
    except:
        print('ERROR! Something went wrong')
        return -1

##################################################################################################################
##################################################################################################################

def normDict(oDict, tol= 0.001):

    '''
    This function takes in a dictionary with species composition information and normalizes the component values to that they sum to 1.0
    If the components are in a percentage it will convert them to fractional values. If the component values are already normalized to
    1.0 in fractional form, the values should not change.

    i.e.  oDict {'CH4': 32, 'O2': 12, 'C3H8': 10, 'N2': 76} ---> {'CH4': 0.246, 'O2': 0.092, 'C3H8': 0.077, 'N2': 0.585}

    Arguments:
        oDict --> [dict] containing the original input dictionary to be normalized

        tol --> [float](optional)[default = 0.001] containing a composition tolerance value. Any values smaller than this in the
                original dictionary will be normalized away and dropped from the output

    Returns:
        nDict --> [dict] containing the now normalized dictionary
    '''

    nDict={}                        # Initializes an empty dictionary to hold the new normalize dictionary values
    factor=1.0/sum(oDict.values())     # Calculates overall scaling factor based upon how much bigger the overall sum is than 1.0 in oDict

    # First round of normalization dropping any values and scaling
    for k in oDict:                    # Iterate through the key values in the original dictionar (oDict)
        if oDict[k]*factor > tol:
            nDict[k] = oDict[k]*factor

    # Second round of normalization to account for any dropped values below the tolerance threshold
    factor=1.0/sum(nDict.values())
    for k in nDict:
        nDict[k] = nDict[k]*factor

    nDict = collections.defaultdict(lambda : 0, nDict)      # Creates defaultdict() object with default values of zeros for any keys
                                                            # not already in nDict
    return nDict

#################################################################################################################################
#################################################################################################################################

def Make_MMIX(Xf, Xa, fuel, ox, inert=None, Xi=None):

    '''
    This fuction generates an overall gas mixture based on the overall fractions of a mixture of fuel gases (Xf), oxidizer (Xa)
    and inert diluents.

    Arguments:
        Xf -->  [float] containing the overall molar fraction of the fuel gas. (i.e. the sum of the individual fuel components)

        Xa -->  [float] containing the overall molar fraction of the oxidizier gas.

        Xi -->  [float](default=None) containing the overall molar fraction of any inert diluent gases.

        fuel -->    [dict] containing the (molar or volume) composition of the fuel gas mixture

                    Example -- {'CH4': 0.25, 'C3H8': 0.25, 'H2': 0.50}

        ox -->  [dict] containing the (molar or volume) composition of the oxidizer gas mixture (probably air)

                Example -- {'N2': 3.76, 'O2': 1}

        inert -->   [dict](default=None) containing the (molar or volume) composition of any inert gases added

    Return:

    '''
    # Normalize input dictionaries to ensure they all sum to one internally before calculations happen
    fuel = normDict(fuel)
    ox = normDict(ox)

    # Test if an actualy argument was passed for [inert] and [Xi]
    if inert and Xi:
        inert = normDict(inert)         # If a value was passed, normalize the inert mixture dictionary
    else:
        inert = collections.defaultdict(lambda: 0)      # If both arguments were not passed, create defaultdict causing all inerts to be zero

    MMIX = collections.defaultdict(lambda : 0)  # Create defaultdict object so that any keys not specifically enumerated will be
                                                # zero without throwing an error if called
    for key in gas_list_all:
        if inert and Xi:
            MMIX[key] = Xf*fuel[key] + Xa*ox[key] + Xi*inert[key]
        else:
            MMIX[key] = Xf*fuel[key] + Xa*ox[key]
    return MMIX

#####################################################################################################################
#####################################################################################################################


# Import Data Files

*Import data files containing the fuel mixtures, the critical temperature information, and the individual volume LFL values for perspective components*

In [ ]:
## IMPORT SYNTHETIC DATA ##
file_path = select_file()   #Prompt the user for a file selection with a dialog window

# Temporary Hard Coded File Path
# file_name = 'Gas_Mixture_Data.csv'
# file_path = open(file_name)

df_in = pd.read_csv(file_path, index_col=0)  # Read the data from the file into a Pandas DataFrame, skipping the header row

In [5]:
# IMPORT FLAMMABILITY TEMPERATURE CRITERA #
path1 = 'Data_Files/Reference_Data/' + 'TempCriteriaData.xlsx'
df_TempCrit = pd.read_excel(path1, index_col=0)

# IMPORT FLAMMABILITY LIMIT INFORMATION #
path2 = 'Data_files/Reference_Data/' + 'FlammableLimitsData.xlsx'
df_limits = pd.read_excel(path2, index_col=0)

print(df_TempCrit)
print('\n\n')
print(df_limits)

                    Name    Tl    Tu
Formula                             
H2              Hydrogen   629  1192
CO       Carbon Monoxide  1417  1259
CH4              Methane  1480  1730
C2H4            Ethylene  1369  1243
C2H6              Ethane  1602  1413
C3H8             Propane  1509  1226



                    Name  X_lower  X_upper
Formula                                   
H2              Hydrogen     4.00    75.00
CO       Carbon Monoxide    12.00    75.00
CH4              Methane     4.40    16.40
C2H4            Ethylene     2.75    28.60
C2H6              Ethane     3.00    12.40
C3H6           Propylene     2.00    11.10
C3H8             Propane     2.10    10.10
C4H10           n-Butane     1.86     8.41
C6H6             Benzene     1.20     7.80


# Iterative Section

In [6]:
df_out = df_in.copy(deep=True)
lfl_list = []
ufl_list = []
Su_list = []

df_out['Xf_LFL']=None
df_out['Xf_UFL']=None
df_out['Xf_stoich'] = None
df_out['Su_stoich'] = None
# df_out['Su_LFL'] = None

for index, row in tqdm(df_out.iterrows(), total=len(df_out), desc='Processing...'):

    # Create fuel dictionary from the row entries in the dataframe
    fuel = dict(zip(gas_list_fuel, row[gas_list_fuel].fillna(0).values))

    # Use Cantera to find the stoichiometric parameters for the fuel
    gas_st = ct.Solution('gri30.yaml')
    gas_st.set_equivalence_ratio(1.0, fuel, air)     # Set the equivalence ratio to 1.0
    X_st = gas_st[fuel.keys()].X    # Get list of values for fuel gases
    Xf_st = sum(X_st)               # Sum individual fuel species to get total molar fuel fraction

    # Calculate the lower and upper flammability limits with Le Chatelier's Method
    lower, upper = Average_Limits(fuel)
    lower = lower/100       # Divide output by 100 to convert to fraction
    upper = upper/100       # Divide output by 100 to convert to fraction
    lfl_list.append(lower)
    ufl_list.append(upper)

    df_out.at[index, 'LFL'] = lower
    df_out.at[index, 'UFL'] = upper

    # Calculate the flame speed at stoichiometric conditions
    totalMix = Make_MMIX(Xf_st, 1-Xf_st, fuel, air)
    Su = Flamespeed(totalMix, Ti, Pi)           # Call Flamespeed() function to iteratively calculate flame speed for the stoichiometric mixture
    Su_list.append(Su)
    df_out.at[index, 'Su_stoich'] = Su

    print(f'Mixture: {index}; LFL: {lower}; UFL: {upper}')



# df_out['LFL'] = lfl_list
# df_out['UFL'] = ufl_list

Processing...:   0%|          | 0/2000 [00:00<?, ?it/s]


************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.602e-05      5.582
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      1.14e-05      6.574
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0002923       5.17
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.004994      2.278
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps         0.192     0.3083
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps          3.28      1.447
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps         24.91     0.9993
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point

# Save Data

In [ ]:
# Generate a timestamped filename to save the mixtures as a CSV
current = datetime.now()                        # Get the current date and time
cString = current.strftime('%m%d_%H%M%S')       # Generate a string from date and time in format [mmdd_hrminsec]
file_name = f'{len(mixDF)}_Mixtures_' + cString + '.csv'      # Concantenate base name with datetime string
data_folder = 'Data_Files'
df_out.to_csv(f'{data_folder}/{file_name}')

In [ ]:
# date_string = datetime.now().strftime("%Y-%m-%d")
# samples = len(df_out)
# save_name = f'{samples}_Samples-' + date_string + '.csv'
# df_out.to_csv(save_name)

In [ ]:
# df_out['UFL'].min()

0.1287981310887235

# Validation Cases

*As a check that all of the functions and methods defined above are working properly, run a set of pure gases or known mixtures through the system to check if the results match with tabulated values in the literature*

# Pure Methane

In [ ]:
## Pure Methane

## Pure Hydrogen

## Pure Propane

# Appendix

In [ ]:
DFtestImport = pd.read_csv('10000_Samples-2023-11-16.csv')

In [ ]:
DFtestImport

,Unnamed: 0,CO,H2,CH4,C2H6,C3H8,LFL,UFL
0,0,0.296011,0.134990,0.002295,0.279019,0.287685,0.055434,0.387282
1,1,0.248707,0.286868,0.035855,0.327413,0.101157,0.054844,0.458377
2,2,0.012814,0.328003,0.143918,0.259185,0.256081,0.034143,0.337218
3,3,0.156798,0.193250,0.120762,0.015430,0.513761,0.043111,0.336144
4,4,0.365782,0.024475,0.205120,0.176616,0.228007,0.063985,0.371261
...,...,...,...,...,...,...,...,...
9995,9995,0.318071,0.180357,0.100274,0.177976,0.223323,0.059824,0.434890
9996,9996,0.271265,0.439603,0.191297,0.073686,0.024149,0.061271,0.576100
9997,9997,0.195887,0.005744,0.217744,0.252358,0.328268,0.047781,0.251380
9998,9998,0.237248,0.202291,0.139809,0.290870,0.129781,0.054165,0.401759


In [ ]:
fuel

NameError: name 'fuel' is not defined

In [ ]:
# Use Cantera to find the stoichiometric parameters for the fuel
gas_st = ct.Solution('gri30.yaml')
gas_st.set_equivalence_ratio(1.0, fuel, air)     # Set the equivalence ratio to 1.0

X_st = gas_st[fuel.keys()].X
Xf_st = sum(X_st)

In [ ]:
Xf_st

0.07383895079058662

In [ ]:
totalMix = Make_MMIX(Xf_st, 1-Xf_st, fuel, air)

In [ ]:
# try:
gas2 = ct.Solution('gri30.yaml')
gas2.X = totalMix
gas2.TP = Ti, Pi

#Laminar Flame Speed
f = ct.FreeFlame(gas2, width=5) #A freely-propagating flat flame
#Energy equation enabled
f.energy_enabled = True
f.set_max_time_step(50000)
f.set_refine_criteria(ratio = 2.0, slope = 0.05, curve = 0.05)
f.transport_model = 'Mix' # Solve with multi or mix component transport properties
f.solve(loglevel=1,auto=True,refine_grid=True, quiet) #
print('Flamespeed Done! ', f.velocity[0])
# except:
#     print('ERROR! Something went wrong')


************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.602e-05      5.501
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     8.553e-06       6.67
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     4.871e-05      5.874
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.001248      4.229
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps       0.03199     0.9579
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps          1.23     0.1558
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps          9.34     0.2108
Attempt Newton solution of steady-state problem...    success.

Problem solved on [9] point

In [ ]:
f.velocity[0]

0.717957070775794

In [ ]:
Xf_st

0.07383895079058662

In [ ]:
key_list = list(fuel.keys())

Tui = [df_limits.loc[key]['X_upper'] for key in key_list]
Tli = [df_limits.loc[key]['X_lower'] for key in key_list]

# Tui = [df_TempCrit.loc[key]['Tu'] for key in list(mixx.keys())]     # Generate list of individual lower critical temperatures
# Tli = [df_TempCrit.loc[key]['Tl'] for key in list(mixx.keys())]     # Generate list of individual upper critical temperatures

df_limits

,Name,X_lower,X_upper
Formula,,,
H2,Hydrogen,4.00,75.00
CO,Carbon Monoxide,12.00,75.00
CH4,Methane,4.40,16.40
C2H4,Ethylene,2.75,28.60
C2H6,Ethane,3.00,12.40
C3H6,Propylene,2.00,11.10
C3H8,Propane,2.10,10.10
C4H10,n-Butane,1.86,8.41
C6H6,Benzene,1.20,7.80


In [ ]:
Tli

[4.0, 12.0, 4.4, 3.0, 2.1]

In [ ]:
key_list

['H2', 'CO', 'CH4', 'C2H6', 'C3H8']

In [ ]:
value_list = list(fuel.values())

np.dot(value_list, Tli)

5.543386624400123

In [ ]:
row = df_in.iloc[0]
fuel = dict(zip(gas_list_fuel, row[gas_list_fuel].fillna(0).values))
fuel

{'CO': 0.2960111661237538,
 'H2': 0.1349902150621418,
 'CH4': 0.0022945322316353,
 'C2H6': 0.2790191633601446,
 'C3H8': 0.2876849232223243}

In [ ]:
Tu, Tl = Tblend(fuel)

In [ ]:
lower_limit({'CH4': 1.0}, air)

1.0
0
0.5
1
0.25
2
0.125
3
0.0625
4
0.03125
5
0.015625
6


(0.001638592121649074, 0.0009118260348097706, 0.0078125)

In [ ]:
gas_test = ct.Solution('gri30.yaml')
gas_test.set_equivalence_ratio(1.0, fuel, air)
Xf_stoich = 1 - sum(gas_test['O2','N2'].X)     # Subtract oxygen and nitrogen (air) to find the molar fuel fraction
Yf_stoich = 1 - sum(gas_test['O2','N2'].Y)     # Subtract oxygen and nitrogen (air) to find the mass fuel fraction

Xf_stoich

0.07383895079058678

In [ ]:
 # Initialize Cantera objects
carbon = ct.Solution('graphite.yaml')        # Create carbon phase object to represent soot
gas1 = ct.Solution('gri30.yaml')             # Create gas phase object to hold species of interest
# gas1.X = fuel            # Set the composition of the gas phase object
# gas1.TP = Ti, Pi         # Set the temperature and pressure of the gas phase object to ambient

phi = 1.0               # Set starting equivalence ratio value

gas1.set_equivalence_ratio(phi, fuel, air)      # Have Cantera automatically set the equivalence ratio to 1.0

Xf_stoich = 1 - sum(gas1['O2','N2'].X)     # Subtract oxygen and nitrogen (air) to find the molar fuel fraction
Yf_stoich = 1 - sum(gas1['O2','N2'].Y)     # Subtract oxygen and nitrogen (air) to find the mass fuel fraction

phi_lfl = phi       # Initialize phi for lower flammability limit
Xf_lfl = Xf_stoich  # Initialize Xf for lower flammability limit

flag = False        # Initialize flammability limit found flag

# Start binary search
# while (flag != True):
gas1.set_equivalence_ratio(phi, fuel, air)

phi_lfl = phi
Xf_lfl = 1 - sum(gas1['O2','N2'].X)     # Subtract oxygen and nitrogen (air) to find the molar fuel fraction
Yf_lfl = 1 - sum(gas1['O2','N2'].Y)     # Subtract oxygen and nitrogen (air) to find the mass fuel fraction

gas1.equilibrate('HP')
t_eq = gas1.T
d_phi = (phi/2)
if t_eq >= Tl:

    phi = phi - d_phi

if t_eq < Tl:

    phi = phi + d_phi

if d_phi < phi_tol:

    flag = True

In [ ]:
phi

0.5

In [ ]:
sum(gas1[gas_list_fuel].X)

0.05898117649912129

In [ ]:
# Comparison with Methane
gas2 = ct.Solution('gri30.yaml')
gas2.set_equivalence_ratio(1.0, 'CH4', air)

gas2['CH4'].X.item()

0.09505703422053233

In [ ]:
# Comparison with Hydrogen
gas3 = ct.Solution('gri30.yaml')
gas3.set_equivalence_ratio(1.0, 'H2', air)

gas3['H2'].X.item()

0.2958579881656805

In [ ]:
# Comparison with Hydrogen
gas4 = ct.Solution('gri30.yaml')
gas4.set_equivalence_ratio(1.0, 'C3H8', air)

gas4['C3H8'].X.item()

0.04032258064516129

In [ ]:
alpha = list(fuel.values())
Tui = [df_TempCrit.loc[key]['Tu'] for key in gas_list_fuel]
Tli = [df_TempCrit.loc[key]['Tl'] for key in gas_list_fuel]
Tl = np.dot(alpha, Tli)
Tu = np.dot(alpha, Tui)

In [ ]:
df_in

,CO,H2,CH4,C2H6,C3H8
0,0.296011,0.134990,0.002295,0.279019,0.287685
1,0.248707,0.286868,0.035855,0.327413,0.101157
2,0.012814,0.328003,0.143918,0.259185,0.256081
3,0.156798,0.193250,0.120762,0.015430,0.513761
4,0.365782,0.024475,0.205120,0.176616,0.228007
...,...,...,...,...,...
95,0.259923,0.143597,0.167871,0.415356,0.013253
96,0.189215,0.229383,0.355587,0.005768,0.220047
97,0.128228,0.286741,0.197178,0.157666,0.230187
98,0.256206,0.259094,0.187128,0.101421,0.196152


In [ ]:
Tl


1388.8578242197054